In [ ]:
!git clone https://github.com/Q-Future/Q-Align.git
%cd Q-Align
!pip install -e ".[train]"
!pip install flash_attn --no-build-isolation

Cloning into 'Q-Align'...
remote: Enumerating objects: 745, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 745 (delta 3), reused 2 (delta 1), pack-reused 732
Receiving objects: 100% (745/745), 42.06 MiB | 19.55 MiB/s, done.
Resolving deltas: 100% (440/440), done.
Updating files: 100% (177/177), done.
/workspace/Q-Align


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Obtaining file:///workspace/Q-Align
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.9/809.9 kB 42.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.7/137.7 kB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.

In [ ]:
!pip install deepspeed


[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
# Copy JSON file
!cp /workspace/train_split_1.json /workspace/Q-Align/playground/data/ft/

# Create directory for video data
!mkdir -p /workspace/video_data

# Copy video files
!cp /workspace/*.mp4 /workspace/video_data/
# If you have GIF files, use this command as well:
# !cp /content/drive/MyDrive/Q-Align_data/*.gif /content/video_data/

In [ ]:
!ls -l /workspace/Q-Align/playground/data/ft/
!ls -l /workspace/video_data/

total 13689
drwxrwxrwx 2 root root 2000678 Jul 24 00:18 agi
drwxrwxrwx 2 root root 2000106 Jul 24 00:18 agiqa3k
drwxrwxrwx 2 root root 2001283 Jul 24 00:18 cgi
drwxrwxrwx 2 root root 2000169 Jul 24 00:18 csiq
drwxrwxrwx 2 root root 2000211 Jul 24 00:18 live
drwxrwxrwx 2 root root 2000250 Jul 24 00:18 livec
drwxrwxrwx 2 root root 2000105 Jul 24 00:18 maxwell
-rw-rw-rw- 1 root root   13077 Jul 24 00:31 train_split_1.json
total 39856
-rw-rw-rw- 1 root root 1367318 Jul 24 00:31  Anatomy_Bad.mp4
-rw-rw-rw- 1 root root 1730556 Jul 24 00:31  Beach_Good.mp4
-rw-rw-rw- 1 root root  710597 Jul 24 00:31  BeachwithProduct_Good.mp4
-rw-rw-rw- 1 root root 2548897 Jul 24 00:31  Building_Bad.mp4
-rw-rw-rw- 1 root root 1551262 Jul 24 00:31  Building_Color_Bad.mp4
-rw-rw-rw- 1 root root 1555896 Jul 24 00:31  Building_Color_worse.mp4
-rw-rw-rw- 1 root root 4063180 Jul 24 00:31  Building_Very_Bad.mp4
-rw-rw-rw- 1 root root  826721 Jul 24 00:31  ExtraHand_Bad.mp4
-rw-rw-rw- 1 root root  238587 Jul 24 00:31

In [ ]:
%%writefile scripts/my_videos_lora.sh
#!/bin/bash

# Create input file with choice
echo "2" > input.txt
export WANDB_API_KEY="d748b17184ce7ec9c05419bb670fbda0fd136a13"

wandb login $WANDB_API_KEY

# Run the script with input redirection
deepspeed q_align/train/train_mem.py < input.txt \
    --deepspeed ./scripts/zero3.json \
    --model_name_or_path q-future/one-align \
    --version v1 \
    --data_path /workspace/Q-Align/playground/data/ft/train_split_1.json \
    --image_folder /workspace/video_data \
    --bf16 True \
    --output_dir /workspace/Q-Align/checkpoints/ \
    --num_train_epochs 1 \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_step 4 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 50000 \
    --save_total_limit 1 \
    --learning_rate 2e-5 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --tf32 True \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --dataloader_num_workers 4 \
    --lazy_preprocess True \
    --report_to wandb


Writing scripts/my_videos_lora.sh


In [ ]:
!chmod +x scripts/my_videos_lora.sh
!bash scripts/my_videos_lora.sh

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
[2024-07-24 00:31:24,117] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-07-24 00:31:26,243] [WARNING] [runner.py:196:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2024-07-24 00:31:26,243] [INFO] [runner.py:555:main] cmd = /usr/bin/python -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMCwgMV19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None q_align/train/train_mem.py --deepspeed ./scripts/zero3.json --model_name_or_path q-future/one-align --version v1 --data_path /workspace/Q-Align/playground/data/ft/train_split_1.json --image_folder /workspace/video_data --bf16 True --output_dir /workspace/Q-Align/checkpoints/ --num_train_epochs 1 --per_device_train_batch_size 1 --per_device_eval_batch_size 1 --gradient_accumulation_step 4 --evaluation_strategy no --save_strategy steps --s

In [ ]:
import shutil
import os

source_path = "preprocessor_config.json"
destination_path = "/workspace/Q-Align/checkpoints/preprocessor_config.json"

# Ensure the destination directory exists
os.makedirs(os.path.dirname(destination_path), exist_ok=True)

# Copy the file
shutil.copy2(source_path, destination_path)

print(f"File copied from {source_path} to {destination_path}")

File copied from preprocessor_config.json to /workspace/Q-Align/checkpoints/preprocessor_config.json


In [ ]:
!ls /workspace/Q-Align/checkpoints/

config.json			  pytorch_model.bin.index.json
preprocessor_config.json	  special_tokens_map.json
pytorch_model-00001-of-00004.bin  tokenizer.model
pytorch_model-00002-of-00004.bin  tokenizer_config.json
pytorch_model-00003-of-00004.bin  trainer_state.json
pytorch_model-00004-of-00004.bin  training_args.bin


In [ ]:
!pwd


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/workspace/Q-Align


In [ ]:
import sys
sys.path.append('/workspace/Q-Align')
from q_align.evaluate.vqa_eval import main
from types import SimpleNamespace

args = SimpleNamespace(
    model_path="/workspace/Q-Align/checkpoints",
    model_base=None,  # Set this to None if loading a full model
    device="cuda:0",
    conv_mode=None,
    temperature=0.2,
    max_new_tokens=512,
    load_8bit=False,
    load_4bit=False,
    debug=False,
    image_aspect_ratio='pad'
)

main(args)

# [
# {
#     "img_path": "i2v.mp4",
#     "gt_score": 3
#   },
#   {
#     "img_path": "i2v (17).mp4",
#     "gt_score": 1
#   },
#   {
#     "img_path": "i2v (20).mp4",
#     "gt_score": 1
#   },
#   {
#     "img_path": "i2v (8).mp4",
#     "gt_score": 1
#   }
# ]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

['good', 'poor', 'high', 'fair', 'low', 'excellent', 'bad', 'fine', 'moderate', 'decent', 'average', 'medium', 'acceptable']
[1781, 6460, 1880, 6534, 4482, 15129, 4319, 2691, 17768, 27189, 6588, 18350, 22691]


Evaluating [test_custom_videos.json]: 100%|██████████| 4/4 [00:00<00:00,  4.93it/s]


Spearmanr 0.7745966692414834 Pearson 0.6473548215720712


In [ ]:
import sys
sys.path.append('/workspace/Q-Align')
from q_align.evaluate.vqa_eval import main
from types import SimpleNamespace

args = SimpleNamespace(
    model_path="/workspace/Q-Align/checkpoints",
    model_base=None,  # Set this to None if loading a full model
    device="cuda:0",
    conv_mode=None,
    temperature=0.2,
    max_new_tokens=512,
    load_8bit=False,
    load_4bit=False,
    debug=False,
    image_aspect_ratio='pad'
)

main(args)
# [
# {
#     "img_path": "i2v.mp4",
#     "gt_score": 2
#   },
#   {
#     "img_path": "i2v (17).mp4",
#     "gt_score": 0.7
#   },
#   {
#     "img_path": "i2v (20).mp4",
#     "gt_score": 1
#   },
#   {
#     "img_path": "i2v (8).mp4",
#     "gt_score": 1
#   }
# ]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

['good', 'poor', 'high', 'fair', 'low', 'excellent', 'bad', 'fine', 'moderate', 'decent', 'average', 'medium', 'acceptable']
[1781, 6460, 1880, 6534, 4482, 15129, 4319, 2691, 17768, 27189, 6588, 18350, 22691]


Evaluating [test_custom_videos.json]: 100%|██████████| 4/4 [00:00<00:00,  4.95it/s]


Spearmanr 0.9486832980505139 Pearson 0.7544963861963276


In [ ]:
import sys
sys.path.append('/workspace/Q-Align')
from q_align.evaluate.vqa_eval import main
from types import SimpleNamespace

args = SimpleNamespace(
    model_path="/workspace/Q-Align/checkpoints",
    model_base=None,  # Set this to None if loading a full model
    device="cuda:0",
    conv_mode=None,
    temperature=0.2,
    max_new_tokens=512,
    load_8bit=False,
    load_4bit=False,
    debug=False,
    image_aspect_ratio='pad'
)

main(args)

# [
# {
#     "img_path": "Mask_good.mp4",
#     "gt_score": 1
#   },
#   {
#     "img_path": "SpiderMan_Bad.mp4",
#     "gt_score": 0.6
#   },
#   {
#     "img_path": "SpiderMan_Bad.mp4",
#     "gt_score": 5
#   },
#   {
#     "img_path": "Mask_good.mp4",
#     "gt_score": 5
#   }
# ]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

['good', 'poor', 'high', 'fair', 'low', 'excellent', 'bad', 'fine', 'moderate', 'decent', 'average', 'medium', 'acceptable']
[1781, 6460, 1880, 6534, 4482, 15129, 4319, 2691, 17768, 27189, 6588, 18350, 22691]


Evaluating [test_custom_videos.json]: 100%|██████████| 4/4 [00:00<00:00,  5.61it/s]

Spearmanr 0.23570226039551587 Pearson 0.04751143381455797


In [ ]:
import os
print(os.listdir('/workspace/Q-Align/checkpoints'))


['q-align-my-videos-lora', 'trainer_state.json', 'config.json', 'pytorch_model-00001-of-00004.bin', 'pytorch_model-00002-of-00004.bin', 'pytorch_model-00003-of-00004.bin', 'pytorch_model-00004-of-00004.bin', 'pytorch_model.bin.index.json', 'tokenizer_config.json', 'special_tokens_map.json', 'tokenizer.model', 'training_args.bin', 'preprocessor_config.json']


In [ ]:
import sys
sys.path.append('/workspace/Q-Align')
from q_align.evaluate.vqa_eval import main
from types import SimpleNamespace

args = SimpleNamespace(
    model_path="/workspace/Q-Align/checkpoints",
    model_base=None,  # Set this to None if loading a full model
    device="cuda:0",
    conv_mode=None,
    temperature=0.2,
    max_new_tokens=512,
    load_8bit=False,
    load_4bit=False,
    debug=False,
    image_aspect_ratio='pad'
)

main(args)

# [

#   {
#     "img_path": "SpiderMan_Bad.mp4",
#     "gt_score": 2
#   },

#   {
#     "img_path": "Mask_good.mp4",
#     "gt_score": 5
#   }
# ]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

['good', 'poor', 'high', 'fair', 'low', 'excellent', 'bad', 'fine', 'moderate', 'decent', 'average', 'medium', 'acceptable']
[1781, 6460, 1880, 6534, 4482, 15129, 4319, 2691, 17768, 27189, 6588, 18350, 22691]


Evaluating [test_custom_videos.json]: 100%|██████████| 2/2 [00:00<00:00,  6.77it/s]

Spearmanr 0.9999999999999999 Pearson 1.0


In [ ]:
%%writefile /workspace/Q-Align/q_align/evaluate/vqa_eval.py
import argparse
import torch

from q_align.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from q_align.conversation import conv_templates, SeparatorStyle
from q_align.model.builder import load_pretrained_model
from q_align.mm_utils import process_images, tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria

from PIL import Image

import requests
from PIL import Image
from io import BytesIO
from transformers import TextStreamer


from scipy.stats import spearmanr, pearsonr

import json
from tqdm import tqdm
from collections import defaultdict

import os

def wa5(logits):
    import numpy as np
    logprobs = np.array([logits["excellent"], logits["good"], logits["fair"], logits["poor"], logits["bad"]])
    probs = np.exp(logprobs) / np.sum(np.exp(logprobs))
    return np.inner(probs, np.array([1,0.75,0.5,0.25,0.]))



def disable_torch_init():
    """
    Disable the redundant torch default initialization to accelerate model creation.
    """
    import torch
    setattr(torch.nn.Linear, "reset_parameters", lambda self: None)
    setattr(torch.nn.LayerNorm, "reset_parameters", lambda self: None)


def load_video(video_file):
    from decord import VideoReader
    vr = VideoReader(video_file)

    # Get video frame rate
    fps = vr.get_avg_fps()

    # Calculate frame indices for 1fps
    frame_indices = [int(fps * i) for i in range(int(len(vr) / fps))]
    frames = vr.get_batch(frame_indices).asnumpy()
    return [Image.fromarray(frames[i]) for i in range(int(len(vr) / fps))]


def main(args):
    # Model
    disable_torch_init()

    model_name = get_model_name_from_path(args.model_path)
    tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name, args.load_8bit, args.load_4bit, device=args.device)


    import json


    image_paths = [
        "/workspace/video_data/"
        # "playground/data/",
        # "playground/data/",
        # "playground/data/KoNViD_1k_videos/",
        # "playground/data/maxwell/",

    ]

    json_prefix = "playground/data/test_jsons/"
    jsons = [

        json_prefix + "test_2.json"
        # "test_lsvq.json",
        # json_prefix + "test_lsvq_1080p.json",
        # json_prefix + "konvid.json",
        # json_prefix + "maxwell_test.json",
    ]

    os.makedirs(f"results/{args.model_path}/", exist_ok=True)


    conv_mode = "mplug_owl2"

    inp = "How would you rate the quality of this video?"

    conv = conv_templates[conv_mode].copy()
    inp =  inp + "\n" + DEFAULT_IMAGE_TOKEN
    conv.append_message(conv.roles[0], inp)
    image = None

    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt() + " The quality of the video is"

    toks = ["good", "poor", "high", "fair", "low", "excellent", "bad", "fine", "moderate",  "decent", "average", "medium", "acceptable"]
    print(toks)
    ids_ = [id_[1] for id_ in tokenizer(toks)["input_ids"]]
    print(ids_)

    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(args.device)

    for image_path, json_ in zip(image_paths, jsons):
        with open(json_) as f:
            iqadata = json.load(f)
            prs, gts = [], []
            for i, llddata in enumerate(tqdm(iqadata, desc="Evaluating [{}]".format(json_.split("/")[-1]))):
                try:
                    try:
                        filename = llddata["img_path"]
                    except:
                        filename = llddata["image"]
                    llddata["logits"] = defaultdict(float)

                    image = load_video(image_path + filename)
                    def expand2square(pil_img, background_color):
                            width, height = pil_img.size
                            if width == height:
                                return pil_img
                            elif width > height:
                                result = Image.new(pil_img.mode, (width, width), background_color)
                                result.paste(pil_img, (0, (width - height) // 2))
                                return result
                            else:
                                result = Image.new(pil_img.mode, (height, height), background_color)
                                result.paste(pil_img, ((height - width) // 2, 0))
                                return result
                    image = [expand2square(img, tuple(int(x*255) for x in image_processor.image_mean)) for img in image]
                    image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'].half().to(args.device)

                    if True:
                        with torch.inference_mode():
                            output_logits = model(input_ids,
                                images=[image_tensor])["logits"][:,-1]
                            for tok, id_ in zip(toks, ids_):
                                llddata["logits"][tok] += output_logits.mean(0)[id_].item()
                            llddata["score"] = wa5(llddata["logits"])
                            # print(llddata)
                            prs.append(llddata["score"])

                            gts.append(llddata["gt_score"])
                            print(llddata)
                            json_ = json_.replace("combined/", "combined-")
                            with open(f"results/{args.model_path}/{json_.split('/')[-1]}", "a") as wf:
                                json.dump(llddata, wf)

                    if i > 0 and i % 200 == 0:
                        print(spearmanr(prs,gts)[0], pearsonr(prs,gts)[0])
                except:
                    continue
            print("Spearmanr", spearmanr(prs,gts)[0], "Pearson", pearsonr(prs,gts)[0])


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--model-path", type=str, default="q-future/one-align")
    parser.add_argument("--model-base", type=str, default=None)
    parser.add_argument("--device", type=str, default="cuda:0")
    parser.add_argument("--conv-mode", type=str, default=None)
    parser.add_argument("--temperature", type=float, default=0.2)
    parser.add_argument("--max-new-tokens", type=int, default=512)
    parser.add_argument("--load-8bit", action="store_true")
    parser.add_argument("--load-4bit", action="store_true")
    parser.add_argument("--debug", action="store_true")
    parser.add_argument("--image-aspect-ratio", type=str, default='pad')
    args = parser.parse_args()
    main(args)


Overwriting /workspace/Q-Align/q_align/evaluate/vqa_eval.py
